In [1]:
import pandas as pd 
import numpy as np
from scipy.optimize import linprog 

In [2]:
data = pd.read_csv('data.csv', sep=';')

In [3]:
data.head(5)

,Product Name,Production Unit Name,Currency,Production Plan,Planned Sale price,Price - Model Curr(USD),Material Cost,Marginal profit per unit,Total marginal profit,Min production lot restrictions,Contract minimum,Market maximum
0,"Candies ""Hazelnuts"". 0.25 kg",Candies,TRY,8200.0,31.406,2.293,1.335,0.958,7856.980,100,100,15000
1,"Candies ""Hazelnuts"". 0.25 kg",Candies,EUR,100.0,2.021,2.284,1.335,0.950,94.959,100,0,10000
2,"Candies ""Hazelnuts"". 0.25 kg",Candies,USD,100.0,2.283,2.283,1.335,0.948,94.834,100,100,5000
3,"Candies ""Cashews"". 0.25 kg",Candies,TRY,1500.0,45.777,3.342,1.925,1.417,2125.330,100,150,1500
4,"Candies ""Cashews"". 0.25 kg",Candies,EUR,800.0,2.946,3.329,1.925,1.404,1123.500,100,0,800


In [4]:
sum_total_marginal_profit = sum(data['Total marginal profit'])
print(sum_total_marginal_profit)

38158.94800000001


In [5]:
data.dtypes

Product Name                        object
Production Unit Name                object
Currency                            object
Production Plan                    float64
Planned Sale price                 float64
Price -  Model Curr(USD)           float64
Material Cost                      float64
Marginal profit per unit           float64
Total marginal profit              float64
Min production lot restrictions      int64
Contract minimum                     int64
Market maximum                       int64
dtype: object

In [6]:
sum_candies = sum(data[data['Production Unit Name'] == 'Candies']['Production Plan'])
sum_lolipops = sum(data[data['Production Unit Name'] == 'Lollipops']['Production Plan'])
sum_dragee = sum(data[data['Production Unit Name'] == 'Dragee']['Production Plan'])

In [7]:
sum_candies, sum_lolipops, sum_dragee

(12000.0, 5000.0, 9999.996000000001)

## Candies

In [8]:
data_1 = data[data['Production Unit Name'] == 'Candies']
MarginPPU = data_1['Marginal profit per unit']

In [9]:
data_1

,Product Name,Production Unit Name,Currency,Production Plan,Planned Sale price,Price - Model Curr(USD),Material Cost,Marginal profit per unit,Total marginal profit,Min production lot restrictions,Contract minimum,Market maximum
0,"Candies ""Hazelnuts"". 0.25 kg",Candies,TRY,8200.0,31.406,2.293,1.335,0.958,7856.980,100,100,15000
1,"Candies ""Hazelnuts"". 0.25 kg",Candies,EUR,100.0,2.021,2.284,1.335,0.950,94.959,100,0,10000
2,"Candies ""Hazelnuts"". 0.25 kg",Candies,USD,100.0,2.283,2.283,1.335,0.948,94.834,100,100,5000
3,"Candies ""Cashews"". 0.25 kg",Candies,TRY,1500.0,45.777,3.342,1.925,1.417,2125.330,100,150,1500
4,"Candies ""Cashews"". 0.25 kg",Candies,EUR,800.0,2.946,3.329,1.925,1.404,1123.500,100,0,800
5,"Candies ""Cashews"". 0.25 kg",Candies,USD,1000.0,3.327,3.327,1.925,1.403,1402.560,100,200,1000
6,"Candies ""Pistachios"". 0.25 kg",Candies,TRY,100.0,48.719,3.557,3.307,0.249,24.944,100,50,1500
7,"Candies ""Pistachios"". 0.25 kg",Candies,EUR,100.0,3.136,3.543,3.307,0.236,23.613,100,0,900
8,"Candies ""Pistachios"". 0.25 kg",Candies,USD,100.0,3.541,3.541,3.307,0.234,23.419,100,50,2000


In [29]:
c = -data_1['Production Plan']
c = pd.to_numeric(c)

In [132]:
B = []
for b in range(len(data_1)):
    B.append(max(np.array(data_1['Min production lot restrictions'])[b], np.array(data_1['Contract minimum'])[b]))
B = pd.Series(B)

bounds = [(b, None) for b in B]

In [12]:
-MarginPPU

0   -0.958
1   -0.950
2   -0.948
3   -1.417
4   -1.404
5   -1.403
6   -0.249
7   -0.236
8   -0.234
Name: Marginal profit per unit, dtype: float64

In [13]:
A_ub = np.diag(MarginPPU)

In [20]:
A_ub

array([[0.958, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.95 , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.948, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 1.417, 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 1.404, 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 1.403, 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.249, 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.236, 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.234]])

In [14]:
b_ub = np.array(data_1['Market maximum'])

In [21]:
b_ub

array([15000, 10000,  5000,  1500,   800,  1000,  1500,   900,  2000],
      dtype=int64)

In [95]:
A_eq = np.ones(len(data_1)).reshape(1, -1)

In [96]:
A_eq

array([[1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [97]:
b_eq = 12000

In [98]:
bounds

[(100, None),
 (100, None),
 (100, None),
 (150, None),
 (100, None),
 (200, None),
 (100, None),
 (100, None),
 (100, None)]

In [102]:
res_1 = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='revised simplex')

In [103]:
res_1

     con: array([0.])
     fun: -91165000.0
 message: 'Optimization terminated successfully.'
     nit: 1
   slack: array([4414.1 , 9905.  , 4905.2 , 1287.45,  659.6 ,  719.4 , 1475.1 ,
        876.4 , 1976.6 ])
  status: 0
 success: True
       x: array([11050.,   100.,   100.,   150.,   100.,   200.,   100.,   100.,
         100.])

## Lollipops


In [142]:
data_2 = data[data['Production Unit Name'] == 'Lollipops']
MarginPPU_2 = data_2['Marginal profit per unit']
c_2= -data_2['Production Plan']
c_2 = pd.to_numeric(c_2)
B_2 = []
for b in range(len(data_2)):
    B_2.append(max(np.array(data_2['Min production lot restrictions'])[b], np.array(data_2['Contract minimum'])[b]))
B_2 = pd.Series(B_2)

bounds_2 = [(b, None) for b in B_2]

A_ub_2 = np.diag(MarginPPU_2)
b_ub_2 = np.array(data_2['Market maximum'])
A_eq_2 = np.ones(len(data_2)).reshape(1, -1)
b_eq_2 = 5000
res_2 = linprog(c_2, A_ub=A_ub_2, b_ub=b_ub_2, A_eq=A_eq_2, b_eq=b_eq_2, bounds=bounds_2, method='revised simplex')

In [143]:
res_2

     con: array([0.])
     fun: -8279484.436387047
 message: 'Optimization terminated successfully.'
     nit: 4
   slack: array([0.00000000e+00, 0.00000000e+00, 1.13686838e-13, 2.78731658e+03,
       9.13600000e+02, 7.81900000e+02, 2.92010000e+03, 9.60450000e+02,
       4.60550000e+02])
  status: 0
 success: True
       x: array([2427.18446602,  408.83074407,  573.3005733 , 1090.68421661,
        200.        ,  100.        ,  100.        ,   50.        ,
         50.        ])

## Dragee 

In [146]:
data_3 = data[data['Production Unit Name'] == 'Dragee']
MarginPPU_3 = data_3['Marginal profit per unit']
c_3= -data_3['Production Plan']
c_3 = pd.to_numeric(c_3)
B_3 = []
for b in range(len(data_3)):
    B_3.append(max(np.array(data_3['Min production lot restrictions'])[b], np.array(data_3['Contract minimum'])[b]))
B_3 = pd.Series(B_3)

bounds_3 = [(b, None) for b in B_3]

A_ub_3 = np.diag(MarginPPU_3)
b_ub_3 = np.array(data_3['Market maximum'])
A_eq_3 = np.ones(len(data_3)).reshape(1, -1)
b_eq_3 = 10000
res_3 = linprog(c_3, A_ub=A_ub_3, b_ub=b_ub_3, A_eq=A_eq_3, b_eq=b_eq_3, bounds=bounds_3, method='revised simplex')

In [147]:
res_3

     con: array([0.])
     fun: -24848677.436496593
 message: 'Optimization terminated successfully.'
     nit: 3
   slack: array([6128.97223915, 1964.75      ,  964.8       ,    0.        ,
          0.        , 1780.6       ,  127.3       ,  128.6       ,
        228.8       ])
  status: 0
 success: True
       x: array([5414.02484035,   50.        ,   50.        , 2719.85494107,
       1366.12021858,  100.        ,  100.        ,  100.        ,
        100.        ])